In [43]:
import pandas as pd
import numpy as np

In [44]:
# CO2 Emission per year per country
co2 = pd.read_csv('https://raw.githubusercontent.com/TrainingByPackt/Interactive-Data-Visualization-with-Python/master/datasets/co2.csv')
# GDP per year per country
gm = pd.read_csv('https://raw.githubusercontent.com/TrainingByPackt/Interactive-Data-Visualization-with-Python/master/datasets/gapminder.csv')

In [45]:
df_gm = gm[['Country', 'region']].drop_duplicates()

In [46]:
df_w_regions = pd.merge(co2, df_gm, left_on='country', right_on='Country', how='inner')
df_w_regions = df_w_regions.drop('Country', axis='columns')
df_w_regions.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,region
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0637,0.0854,0.154,0.242,0.294,0.412,0.35,0.316,0.299,South Asia
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.2800,1.3000,1.460,1.480,1.560,1.790,1.68,1.730,1.960,Europe & Central Asia
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.9900,3.1900,3.160,3.420,3.300,3.290,3.46,3.510,3.720,Middle East & North Africa
3,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.1000,1.2000,1.180,1.230,1.240,1.250,1.33,1.250,1.290,Sub-Saharan Africa
4,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.9100,5.1400,5.190,5.450,5.540,5.360,5.42,5.360,5.380,America


In [47]:
new_co2 = pd.melt(df_w_regions, id_vars=['country', 'region'])
columns = ['country', 'region', 'year', 'co2']
new_co2.columns=columns
new_co2

,country,region,year,co2
0,Afghanistan,South Asia,1800,NaN
1,Albania,Europe & Central Asia,1800,NaN
2,Algeria,Middle East & North Africa,1800,NaN
3,Angola,Sub-Saharan Africa,1800,NaN
4,Antigua and Barbuda,America,1800,NaN
...,...,...,...,...
37190,Vanuatu,East Asia & Pacific,2014,0.595
37191,Venezuela,America,2014,6.030
37192,Vietnam,East Asia & Pacific,2014,1.800
37193,Zambia,Sub-Saharan Africa,2014,0.288


In [48]:
df_co2 = new_co2[new_co2['year'].astype('int64') > 1963]
df_co2 = df_co2.sort_values(by=['country', 'year'])
df_co2['year'] = df_co2['year'].astype('int64')
df_co2.head()

,country,region,year,co2
28372,Afghanistan,South Asia,1964,0.0863
28545,Afghanistan,South Asia,1965,0.1010
28718,Afghanistan,South Asia,1966,0.1080
28891,Afghanistan,South Asia,1967,0.1240
29064,Afghanistan,South Asia,1968,0.1160


In [49]:
# We'll do the same manipulation for our GDP data
df_gdp = gm[['Country', 'Year', 'gdp']]
df_gdp.columns = ['country', 'year', 'gdp']
df_gdp.head()

,country,year,gdp
0,Afghanistan,1964,1182.0
1,Afghanistan,1965,1182.0
2,Afghanistan,1966,1168.0
3,Afghanistan,1967,1173.0
4,Afghanistan,1968,1187.0


In [50]:
data = pd.merge(df_co2, df_gdp, on=['country', 'year'], how='left')
data = data.dropna()
data.head()

,country,region,year,co2,gdp
0,Afghanistan,South Asia,1964,0.0863,1182.0
1,Afghanistan,South Asia,1965,0.1010,1182.0
2,Afghanistan,South Asia,1966,0.1080,1168.0
3,Afghanistan,South Asia,1967,0.1240,1173.0
4,Afghanistan,South Asia,1968,0.1160,1187.0


In [51]:
# Checking correlation between CO2 and GDP
np.corrcoef(np.array(data['co2']), np.array(data['gdp']))


array([[1.        , 0.78219731],
       [0.78219731, 1.        ]])

In [52]:
# Creating Base Static Plot for Interactivity

In [53]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row

In [54]:
output_notebook()

Loading BokehJS ...

In [55]:
regions_list = data.region.unique().tolist()

In [56]:
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [69]:
source = ColumnDataSource(data={
    'x': data.gdp[data['year']==1964],
    'y': data.co2[data['year']==1964],
    'country': data.country[data['year']==1964],
    'region': data.region[data['year']==1964]
})


In [58]:
xmin, xmax = min(data.gdp), max(data.gdp)
ymin, ymax = min(data.co2), max(data.co2)

In [59]:
plot = figure(title="CO2 Emission VS GDP in 1964",
             plot_height=600, plot_width=1000,
              x_range=(xmin, xmax),
              y_range=(ymin, ymax),
              y_axis_type='log'
             )
plot.circle(x='x', y='y', fill_alpha=0.8, source=source, legend_label='region', color=dict(field='region', transform=color_mapper), size=7)
plot.legend.location = 'bottom_right'
plot.xaxis.axis_label = 'Income Per Person'
plot.yaxis.axis_label = 'CO2 Emissions (tons per person)'
show(plot)

In [60]:
slider = Slider(start=min(data.year), end=max(data.year), step=1, value=min(data.year), title='Year')

In [70]:
def update_plot(attr, old, new):
    yr = slider.value
    new_data = {
        'x': data.gdp[data['year']==yr],
        'y': data.co2[data['year']==yr],
        'country': data.country[data['year']==yr],
        'region': data.region[data['year']==yr],
    }
    source.data = new_data
    plot.title.text = 'CO2 Emissions vs GDP in %d' %yr

In [62]:
slider.on_change('value', update_plot)
layout = row(widgetbox(slider), plot)
curdoc().add_root(layout)

In [63]:
hover = HoverTool(tooltips=[('Country', '@country'), ('GDP', '@x'), ('CO2 Emission', '@y')])
plot.add_tools(hover)